this is for the positional embedding dev, primes and binary. 


In [7]:
import torch
import math
def int_to_binary_tensor(number, max_length):

    binary_string = bin(number)[2:]  # Convert to binary and remove the '0b' prefix.
    same_len = binary_string.zfill(max_length)
    seperated =torch.tensor([float(i) for i in same_len])
    return seperated
def create_binary_tensor(input_length):
    max_length = math.ceil(math.log2(input_length + 1))
    binary_tensors = []

    # Iterate through numbers from 1 to input_length
    for number in range(1, input_length + 1):
        binary_tensor = int_to_binary_tensor(number, max_length)
        #print(binary_tensor.shape)
        binary_tensors.append(binary_tensor)
        
    # Stack the binary tensors to form a 2D tensor
    stacked_tensor = torch.stack(binary_tensors)

    return stacked_tensor

def binary_self_interactions(input_length):
    stacked_tensor = create_binary_tensor(input_length)
    res = torch.zeros((stacked_tensor.shape[0],stacked_tensor.shape[1]**2))
    
    for i in range(stacked_tensor.shape[0]):
        res[i, :] = torch.outer(stacked_tensor[i],stacked_tensor[i]).flatten()
    return res

In [8]:
e = create_binary_tensor(input_length = 512)
print(e.shape)
b = binary_self_interactions(input_length = 512)
print(b.shape)

torch.Size([512, 10])
torch.Size([512, 100])


In [4]:
b.shape

torch.Size([512, 100])

In [36]:
import torch.nn as nn

class BinaryPositionalEmbedding(nn.Module):
    def __init__(self, max_len, embedding_dim):
        super(BinaryPositionalEmbedding, self).__init__()
        
        self.positional_input = binary_self_interactions(max_len)
        
        # Determine positional_emb_dim from positional_input
        positional_emb_dim = self.positional_input.shape[1]
        
        # Linear layer with input size of positional_emb_dim and output size of embedding_dim
        self.linear = nn.Linear(positional_emb_dim, embedding_dim)

    def forward(self, x):
        batch_size = x.shape[0]
        # Apply the linear layer to each position
        x_add = self.linear(self.positional_input)
        x_add = x_add.unsqueeze(0).repeat(batch_size, 1, 1)
        
        
        return x + x_add

In [40]:
b = BinaryPositionalEmbedding(2,3)

In [41]:
i = torch.rand(2,2,3)

In [43]:
b(i).shape

torch.Size([2, 2, 3])